In [1]:
ListaPalavras = ['gato', 'elefante', 'rato', 'rato', 'gato']
palavrasRDD = sc.parallelize(ListaPalavras, 4)
print(type(palavrasRDD))

<class 'pyspark.rdd.RDD'>


In [7]:
# EXERCICIO
def Plural(palavra):
    """Adds an 's' to `palavra`.

    Args:
        palavra (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return '{0}{1}'.format(palavra, 's')

print(Plural('gato'))

gatos


In [8]:
help(Plural)

Help on function Plural in module __main__:

Plural(palavra)
    Adds an 's' to `palavra`.
    
    Args:
        palavra (str): A string.
    
    Returns:
        str: A string with 's' added to it.



In [9]:
assert Plural('rato')=='ratos', 'resultado incorreto!'
print ('OK')

OK


In [18]:
# EXERCICIO
pluralRDD = palavrasRDD.map(Plural)
print (pluralRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [19]:
assert pluralRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'], 'valores incorretos!'
print ('OK')

OK


In [20]:
# EXERCICIO
pluralLambdaRDD = palavrasRDD.map(lambda x: '{0}{1}'.format(x, 's'))
print (pluralLambdaRDD.collect())

['gatos', 'elefantes', 'ratos', 'ratos', 'gatos']


In [21]:
assert pluralLambdaRDD.collect()==['gatos','elefantes','ratos','ratos','gatos'], 'valores incorretos!'
print ('OK')

OK


In [32]:
# EXERCICIO
pluralTamanho = (pluralRDD
                 .map(lambda x: len(x))
                 .collect()
                 )
print (pluralTamanho)

[5, 9, 5, 5, 5]


In [33]:
assert pluralTamanho==[5,9,5,5,5], 'valores incorretos'
print ("OK")

OK


In [36]:
# EXERCICIO
palavraPar = palavrasRDD.map(lambda x: (x, 1))
print (palavraPar.collect())

[('gato', 1), ('elefante', 1), ('rato', 1), ('rato', 1), ('gato', 1)]


In [37]:
assert palavraPar.collect() == [('gato',1),('elefante',1),('rato',1),('rato',1),('gato',1)], 'valores incorretos!'
print ("OK")

OK


In [52]:
# EXERCICIO
palavrasGrupo = palavraPar.groupByKey()
for chave, valor in palavrasGrupo.collect():
    valores = list(valor)
    print ('{}: {}'.format(chave, valores))

rato: [1, 1]
elefante: [1]
gato: [1, 1]


In [44]:
assert sorted(palavrasGrupo.mapValues(lambda x: list(x)).collect()) == [('elefante', [1]), ('gato',[1, 1]), ('rato',[1, 1])], 'Valores incorretos!'
print ("OK")

OK


In [62]:
# EXERCICIO
contagemGroup = palavrasGrupo.map(lambda xy: (xy[0],sum(xy[1])))
print (contagemGroup.collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [63]:
assert sorted(contagemGroup.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [65]:
# EXERCICIO
from operator import add
contagem = palavraPar.reduceByKey(add)
print( contagem.collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [66]:
assert sorted(contagem.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [68]:
# EXERCICIO
contagemFinal = (palavrasRDD
                 .map(lambda x: (x, 1))
                 .reduceByKey(add)              
                 )
print (contagemFinal.collect())

[('rato', 2), ('elefante', 1), ('gato', 2)]


In [69]:
assert sorted(contagemFinal.collect())==[('elefante',1), ('gato',2), ('rato',2)], 'valores incorretos!'
print ("OK")

OK


In [72]:


# EXERCICIO
palavrasUnicas = (contagemFinal
                  .count()
                 )
print (palavrasUnicas)



3


In [73]:
assert palavrasUnicas==3, 'valor incorreto!'
print ("OK")

OK


In [76]:
# EXERCICIO
# add é equivalente a lambda x,y: x+y
from operator import add
total = (contagemFinal
         .map(lambda xy: xy[1])
         .reduce(add)
         )
media = total / float(palavrasUnicas)
print (total)
print (round(media, 2))


5
1.67


In [77]:
assert round(media, 2)==1.67, 'valores incorretos!'
print ("OK")

OK
